In [1]:
import IJulia

# The julia kernel has built in support for Revise.jl, so this is the 
# recommended approach for long-running sessions:
# https://github.com/JuliaLang/IJulia.jl/blob/9b10fa9b879574bbf720f5285029e07758e50a5e/src/kernel.jl#L46-L51

# Users should enable revise within .julia/config/startup_ijulia.jl:
# https://timholy.github.io/Revise.jl/stable/config/#Using-Revise-automatically-within-Jupyter/IJulia-1

# clear console history
IJulia.clear_history()

fig_width = 7
fig_height = 5
fig_format = :retina
fig_dpi = 96

# no retina format type, use svg for high quality type/marks
if fig_format == :retina
  fig_format = :svg
elseif fig_format == :pdf
  fig_dpi = 96
  # Enable PDF support for IJulia
  IJulia.register_mime(MIME("application/pdf"))
end

# convert inches to pixels
fig_width = fig_width * fig_dpi
fig_height = fig_height * fig_dpi

# Intialize Plots w/ default fig width/height
try
  import Plots

  # Plots.jl doesn't support PDF output for versions < 1.28.1
  # so use png (if the DPI remains the default of 300 then set to 96)
  if (Plots._current_plots_version < v"1.28.1") & (fig_format == :pdf)
    Plots.gr(size=(fig_width, fig_height), fmt = :png, dpi = fig_dpi)
  else
    Plots.gr(size=(fig_width, fig_height), fmt = fig_format, dpi = fig_dpi)
  end
catch e
  # @warn "Plots init" exception=(e, catch_backtrace())
end

# Initialize CairoMakie with default fig width/height
try
  import CairoMakie
  
  CairoMakie.activate!(type = string(fig_format))
  CairoMakie.update_theme!(resolution=(fig_width, fig_height))
catch e
    # @warn "CairoMakie init" exception=(e, catch_backtrace())
end
  
# Set run_path if specified
try
  run_path = raw"/home/alexandersmolka/exess/doc/api/base"
  if !isempty(run_path)
    cd(run_path)
  end
catch e
  @warn "Run path init:" exception=(e, catch_backtrace())
end


# emulate old Pkg.installed beahvior, see
# https://discourse.julialang.org/t/how-to-use-pkg-dependencies-instead-of-pkg-installed/36416/9
import Pkg
function isinstalled(pkg::String)
  any(x -> x.name == pkg && x.is_direct_dep, values(Pkg.dependencies()))
end

# ojs_define
if isinstalled("JSON") && isinstalled("DataFrames")
  import JSON, DataFrames
  global function ojs_define(; kwargs...)
    convert(x) = x
    convert(x::DataFrames.AbstractDataFrame) = Tables.rows(x)
    content = Dict("contents" => [Dict("name" => k, "value" => convert(v)) for (k, v) in kwargs])
    tag = "<script type='ojs-define'>$(JSON.json(content))</script>"
    IJulia.display(MIME("text/html"), tag)
  end
elseif isinstalled("JSON")
  import JSON
  global function ojs_define(; kwargs...)
    content = Dict("contents" => [Dict("name" => k, "value" => v) for (k, v) in kwargs])
    tag = "<script type='ojs-define'>$(JSON.json(content))</script>"
    IJulia.display(MIME("text/html"), tag)
  end
else
  global function ojs_define(; kwargs...)
    @warn "JSON package not available. Please install the JSON.jl package to use ojs_define."
  end
end


# don't return kernel dependencies (b/c Revise should take care of dependencies)
nothing


In [2]:
#| echo: false
if !isdefined(Main, :ExESS)
include(joinpath(@__DIR__, "..", "..", "..", "exess.jl", "src", "ExESS.jl"))
using .ExESS
end

println("AVOGADRO_CONSTANT:         $AVOGADRO_CONSTANT \t [m s-1]")
println("BOLTZMANN_CONSTANT         $BOLTZMANN_CONSTANT \t [m s-1]")
println("ELEMENTARY_CHARGE:         $ELEMENTARY_CHARGE \t [m s-1]")
println("GRAVITATIONAL_CONSTANT:    $GRAVITATIONAL_CONSTANT    [m s-1]")
println("PLANCK_CONSTANT:           $PLANCK_CONSTANT           [m s-1]")
println("STEFAN_BOLTZMANN_CONSTANT: $STEFAN_BOLTZMANN_CONSTANT [m s-1]")
println("UNIVERSAL_GAS_CONSTANT:    $UNIVERSAL_GAS_CONSTANT    [m s-1]")
println("")
println("LUNAR_DAY:                 $LUNAR_DAY                 [m s-1]")
println("LUNAR_MASS:                $LUNAR_MASS                [m s-1]")
println("LUNAR_RADIUS:              $LUNAR_RADIUS              [m s-1]")
println("LUNAR_g0:                  $LUNAR_g0                  [m s-1]")
println("LUNAR_vesc:                $LUNAR_vesc                [m s-1]")

AVOGADRO_CONSTANT:         6.02214086e23 	 [m s-1]
BOLTZMANN_CONSTANT         1.38065e-23 	 [m s-1]
ELEMENTARY_CHARGE:         1.602176634e-19 	 [m s-1]
GRAVITATIONAL_CONSTANT:    6.67408e-11    [m s-1]
PLANCK_CONSTANT:           6.62607004e-34           [m s-1]
STEFAN_BOLTZMANN_CONSTANT: 5.67e-8 [m s-1]
UNIVERSAL_GAS_CONSTANT:    8.3144626181532    [m s-1]

LUNAR_DAY:                 2.5488e6                 [m s-1]
LUNAR_MASS:                7.24767309e22                [m s-1]
LUNAR_RADIUS:              1.7374e6              [m s-1]
LUNAR_g0:                  1.6024717046259258                  [m s-1]
LUNAR_vesc:                2359.7179236582847                [m s-1]
